In [4]:
import torch
import torch.nn as nn
from blitz.modules import BayesianLinear
from blitz.utils import variational_estimator
import data_pipes

df_path = "processed_aapl_data.csv"
data = data_pipes.process_df_3(df_path)
x, y, close = data["x"], data["y"], data["close"]
# y = (y - y.min()) / (y.max() - y.min())
# y = y - y.mean()
print(f"""
x shape: {x.shape}
y shape: {y.shape}     
close shape: {close.shape} 
""")


x shape: (2427, 20, 13)
y shape: (2427, 1)     
close shape: (2427, 1) 



In [5]:
@variational_estimator
class BayesianGeorgia(nn.Module):
    def __init__(self, config, win_past=20, features=13, bias=False):
        super().__init__()
        self.layers = nn.ModuleList()
        self.layers.append(nn.Flatten())

        # Construct Bayesian layers with activation and dropout
        for idx in range(len(config['neurons'])):
            # Choose activation function
            activation_fn = {
                'relu': nn.ReLU(),
                'selu': nn.SELU(),
                'sigmoid': nn.Sigmoid(),
                'tanh': nn.Tanh(),
                'none': None
            }.get(config['activations'][idx].lower(), None)

            # Define layer input and output dimensions
            in_features = features * win_past if idx == 0 else config['neurons'][idx - 1]
            out_features = config['neurons'][idx]

            # Replace nn.Linear with BayesianLinear
            self.layers.append(BayesianLinear(in_features, out_features, bias=bias))

            # Add activation if specified
            if activation_fn is not None:
                self.layers.append(activation_fn)

            # Add dropout
            dropout_rate = config['dropouts'][idx]
            self.layers.append(nn.Dropout(dropout_rate))

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x


In [7]:
config = {
    'neurons': [128, 256, 256, 128, 32, 1],
    'activations': ['none', 'selu', 'selu', 'selu', 'selu', 'tanh'],
    'dropouts': [0.3, 0.3, 0.3, 0.3, 0.3, 0.3]
}

inputs = torch.tensor(x, dtype=torch.float32)
targets = torch.tensor(y, dtype=torch.float32)
model = BayesianGeorgia(config)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

criterion = nn.MSELoss()  # or your preferred loss function
kl_weight = 0.1           # regularization strength

epochs = 250
model.train()
for epoch in range(epochs):
    optimizer.zero_grad()

    outputs = model(inputs)
    data_loss = criterion(outputs, targets)

    # Blitz computes complexity (KL-divergence)
    complexity_loss = model.nn_kl_divergence() / len(inputs)

    loss = data_loss + kl_weight * complexity_loss
    loss.backward()
    optimizer.step()

    print(f'Epoch: {epoch}, Loss: {loss.item():.4f}')


Epoch: 0, Loss: 34.5468
Epoch: 1, Loss: 34.5098
Epoch: 2, Loss: 34.4961
Epoch: 3, Loss: 34.4917
Epoch: 4, Loss: 34.5241
Epoch: 5, Loss: 34.4611
Epoch: 6, Loss: 34.4817
Epoch: 7, Loss: 34.4912
Epoch: 8, Loss: 34.5426
Epoch: 9, Loss: 34.4577
Epoch: 10, Loss: 34.4689
Epoch: 11, Loss: 34.4751
Epoch: 12, Loss: 34.4355
Epoch: 13, Loss: 34.4016
Epoch: 14, Loss: 34.4762
Epoch: 15, Loss: 34.4091
Epoch: 16, Loss: 34.4178
Epoch: 17, Loss: 34.3784
Epoch: 18, Loss: 34.3425
Epoch: 19, Loss: 34.3924
Epoch: 20, Loss: 34.4186
Epoch: 21, Loss: 34.4576
Epoch: 22, Loss: 34.3796
Epoch: 23, Loss: 34.3527
Epoch: 24, Loss: 34.3335
Epoch: 25, Loss: 34.2983
Epoch: 26, Loss: 34.4076
Epoch: 27, Loss: 34.3918
Epoch: 28, Loss: 34.3081
Epoch: 29, Loss: 34.3434
Epoch: 30, Loss: 34.2937
Epoch: 31, Loss: 34.3527
Epoch: 32, Loss: 34.3661
Epoch: 33, Loss: 34.3324
Epoch: 34, Loss: 34.3239
Epoch: 35, Loss: 34.2665
Epoch: 36, Loss: 34.2909
Epoch: 37, Loss: 34.3451
Epoch: 38, Loss: 34.2997
Epoch: 39, Loss: 34.3177
Epoch: 40,